
#TP:  KNN sur les données "spam"


> **Groupe-8**


* Tanekeu Bill Jordan-------------------------------19M2459

* Ndongo Ndongo Papia Ange Borel--------19M2308

* Ghomsi Sikali Diane Laure---------------------18U2202

* Tchuimo Wamba Jordan------------------------19M2396 


Données utilisé: SMS Spam Collection Dataset

Lien de téléchargement: 

https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset/download

In [48]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


In [49]:
from google.colab import files
import io
uploader = files.upload()

Saving spam.csv to spam (1).csv


In [50]:
def load_data():
    data = pd.read_csv(io.BytesIO(uploader["spam.csv"]), encoding = "ISO-8859-1")
    data = data.drop(columns=['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'])
    data.columns = ["class", "message"]
    return data

fonction de chargement du jeu de données.
après chargement du fichier .csv , il supprime les trois dernières colonnes qui nous sont inutiles. Ensuite on renomme les deux premières en respectivement  class et message

In [53]:
data.head

<bound method NDFrame.head of      class                                            message
0      ham   go jurong point crazy available bugis n great...
1      ham                            ok lar joking wif u oni
2     spam   free entry 2 wkly comp win fa cup final tkts ...
3      ham                u dun say early hor u c already say
4      ham        nah dont think goes usf lives around though
...    ...                                                ...
5567  spam   2nd time tried 2 contact u u å£750 pound priz...
5568   ham                        ì b going esplanade fr home
5569   ham                        pity mood soany suggestions
5570   ham   guy bitching acted like id interested buying ...
5571   ham                                     rofl true name

[5572 rows x 2 columns]>

In [54]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [56]:
def preprocess_data(data):
    print("Preprocessing data...")
    
    punc = string.punctuation           #Liste de ponctuation 
    sw = stopwords.words('english')     # liste des mots vides
    for i in range(len(data)):
         # Supprime la ponctuation et les symboles courants 
        for item in punc:
            data[i] = data[i].replace(item, "")
         # Mettez toutes les lettres en minuscules et supprimez les mots vides
        splittedWords = data[i].split()
        newText = ""
        for word in splittedWords:
            if word not in sw:
                word = word.lower()
                newText = newText + " " + word  
        data[i] = newText
    return data


In [57]:
df1 = data
data_pre = preprocess_data(df1['message'])
data_pre

Preprocessing data...


0        go jurong point crazy available bugis n great...
1                                 ok lar joking wif u oni
2        free entry 2 wkly comp win fa cup final tkts ...
3                     u dun say early hor u c already say
4             nah dont think goes usf lives around though
                              ...                        
5567     2nd time tried 2 contact u u å£750 pound priz...
5568                          ì b going esplanade fr home
5569                          pity mood soany suggestions
5570     guy bitching acted like id interested buying ...
5571                                       rofl true name
Name: message, Length: 5572, dtype: object

#Séparation données d'entraînement et test

In [67]:
def split_data(data):
    classe = data["class"]
    features = data_pre.to_numpy()
    labels = classe.to_numpy()

    training_data, test_data, training_labels, test_labels = train_test_split(features, labels, test_size = 0.27, random_state = 42)
    return training_data , test_data , training_labels , test_labels



Tout d'abord, il était nécessaire d'avoir les textes des e-mails dans un tableau qui leur était propre et les étiquettes dans un autre tableau qui leur était propre. Ainsi, les textes des e-mails étaient stockés dans des fonctionnalités et les étiquettes étaient stockées dans des étiquettes. La méthode train_test_split a ensuite été utilisée pour diviser les données en training_data, test_data, training_labels et test_labels. L'état aléatoire a été défini sur 42 pour garantir que la même sortie de mélange aléatoire sera obtenue à des fins de test. Après le fractionnement, training_data, test_data, training_labels et test_labels sont renvoyés.


#L'algorithme KNN


In [59]:
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts

Cette fonction prend un seul texte d'e-mail et le divise à l'aide de split(). La fréquence d'occurrence de chaque mot dans l'e-mail est comptée et enregistrée dans wordCounts, qui est de type dictionnaire. Le dictionnaire wordCounts est ensuite renvoyé.

#fonction euclidean_difference()

In [60]:
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
        if word in test_WordCounts and word in training_WordCounts:
            total += (test_WordCounts[word] - training_WordCounts[word])**2
            del training_WordCounts[word]
        else:
            total += test_WordCounts[word]**2
        for word in training_WordCounts:
            total += training_WordCounts[word]**2
     
    return total**0.5

#fonction get_class()

In [61]:
def get_class(selected_Kvalues): 
    spam_count = 0 
    ham_count = 0
    for value in selected_Kvalues:
      if value[0] == "spam":
        spam_count+= 1
      else:
        ham_count+= 1
      if spam_count > ham_count :
        return "spam"
      else:
        return "ham"

    

la fonction précédente permet de déterminer la classe d'un mail.
En effet après avoir obtenu les k voisins les plus proches , on fait une décompte par lot. si parmis ces k plus proches voisins on a plus de spam que de ham alors le mail considéré est un spam. il est un ham dans le cas contraire

fonction knn_classifier()

In [62]:
def knn_classifier(training_data, training_labels, test_data, k , tsize):
  print("Running KNN classifier...")
  result = []
  counter = 1
  training_WordCounts = []
  for training_text in training_data:
    training_WordCounts.append(get_count(training_text))
  for test_text in test_data:
    similarity = []
    test_WordCounts = get_count(test_text)
    for index in range(len(training_data)):
      euclidean_diff =  euclidean_difference(test_WordCounts , training_WordCounts[index])
      similarity.append([training_labels[index], euclidean_diff])
    similarity = sorted(similarity , key = lambda i:i[1])
    selected_Kvalues = []
    for i in range(k) : 
      selected_Kvalues.append(similarity[i])
    result.append(get_class(selected_Kvalues))
  return result


Le classifieur KNN ci dessus traite un jeu de données entier de facon à prendre chaque donnée contenu dans l'ensemble de test et calculer sa distance par rapport à toutes les données de l'ensemble de "train". il stocke les resultats dans un tableau(similarity) puis ce tableau est trié dans l'ordre croissant des distances; en fonctioin du k défini , on consevera les k premières distances qui nous permettront d'évaluer la classe de la donnée de test correspondante.
Chaque résultat est ainsi sauvegardé dans dans un tableau qui est renvoyé à la fin de la procédure.

Main() function

In [68]:
def main(k):
  data = load_data()
  data = preprocess_data(data)
  training_data , test_data , training_labels , test_labels = split_data(data)
  tsize = len(test_data)
  result = knn_classifier(training_data , training_labels , test_data[:tsize] , k , tsize)
  accuracy = accuracy_score(test_labels[:tsize], result)
  print("training data size\t:" + str(len(training_data)))
  print("k value\t\t\t\t:" + str(k))
  print("Samples tested\t\t:" + str(tsize))
  print("% accuracy\t\t\t\t:" + str(accuracy*100))
  print("number correct\t\t:" + str(int(accuracy*tsize)))
  print("Number wrong\t\t:" + str(int(1-accuracy)*tsize))

Notre fonction principale effectue le travail global de KNN sur notre jeu de données entier. En effet on charge d'abord ces données , ensuite on traite les jeu de données avec la fonction preproces_data qui supprime les signes de ponctuations et autres dans les mails. pour poursuivre , on divise le dataset en trois partie comme expliqué au niveau de split_data.  ensuite on affiche la taille des données de test , les résultats de prédictions et les taux de succès , graces aux fonctions écrites plus haut.

In [73]:
main(11)

Preprocessing data...


KeyError: ignored